# Repaso: Ingeniería de Software en Python

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/sonder-art/fdd_p26/blob/main/clase/11_python_intermedio/code/01_repaso_ingenieria.ipynb)

Repaso rápido de los conceptos del curso de DataCamp: paquetes, clases, documentación y testing. Ejecuta cada celda y asegúrate de que entiendes qué pasa.

---
## 1. Paquetes y módulos

Un **módulo** es un archivo `.py`. Un **paquete** es un directorio con `__init__.py`.

Vamos a crear un paquete mínimo directamente en este notebook para ver cómo funciona.

In [ ]:
import os

# Crear la estructura de un paquete
os.makedirs("mi_paquete", exist_ok=True)

# Módulo: operaciones.py
with open("mi_paquete/operaciones.py", "w") as f:
    f.write('''
def sumar(a, b):
    """Suma dos números."""
    return a + b

def restar(a, b):
    """Resta b de a."""
    return a - b
''')

# Módulo: utilidades.py
with open("mi_paquete/utilidades.py", "w") as f:
    f.write('''
def formatear(resultado, decimales=2):
    """Formatea un número con N decimales."""
    return f"{resultado:.{decimales}f}"
''')

# __init__.py: la puerta de entrada
with open("mi_paquete/__init__.py", "w") as f:
    f.write('''
from .operaciones import sumar, restar
from .utilidades import formatear
''')

print("Paquete creado:")
for f in sorted(os.listdir("mi_paquete")):
    print(f"  mi_paquete/{f}")

In [ ]:
# Ahora podemos importar desde el paquete
from mi_paquete import sumar, formatear

resultado = sumar(3.14159, 2.71828)
print(f"sumar(π, e) = {formatear(resultado)}")

# El __init__.py permite esto.
# Sin él, tendrías que hacer:
# from mi_paquete.operaciones import sumar

In [ ]:
# ¿Qué exporta nuestro paquete?
import mi_paquete
print("Contenido público:")
print([x for x in dir(mi_paquete) if not x.startswith("_")])

**Puntos clave:**
- `__init__.py` define qué ve el usuario cuando hace `import mi_paquete`
- Los imports relativos (`.operaciones`) referencian módulos dentro del mismo paquete
- Un buen paquete exporta **solo** lo que el usuario necesita

---
## 2. Clases

Una clase agrupa **datos** (atributos) y **comportamiento** (métodos).

In [ ]:
class Contador:
    """Cuenta eventos por categoría."""

    def __init__(self, nombre):
        self.nombre = nombre
        self._conteos = {}  # _ = uso interno

    def registrar(self, categoria):
        """Registra un evento."""
        self._conteos[categoria] = self._conteos.get(categoria, 0) + 1

    def total(self):
        """Total de eventos registrados."""
        return sum(self._conteos.values())

    def resumen(self):
        """Conteo por categoría."""
        return dict(self._conteos)

# Uso
c = Contador("errores")
c.registrar("404")
c.registrar("500")
c.registrar("404")

print(f"Total: {c.total()}")
print(f"Resumen: {c.resumen()}")

### Herencia: compartir comportamiento

La herencia permite crear clases que **comparten** una base pero **difieren** en detalles.

In [ ]:
class FuenteDatos:
    """Base para cualquier fuente de datos."""

    def __init__(self, nombre):
        self.nombre = nombre

    def leer(self):
        raise NotImplementedError("Cada fuente debe implementar leer()")

    def __repr__(self):
        return f"{self.__class__.__name__}('{self.nombre}')"


class FuenteTexto(FuenteDatos):
    """Lee datos desde un string (para demos)."""

    def __init__(self, nombre, contenido):
        super().__init__(nombre)  # llama al __init__ del padre
        self.contenido = contenido

    def leer(self):
        return self.contenido.strip().split("\n")


class FuenteNumerica(FuenteDatos):
    """Genera una secuencia de números."""

    def __init__(self, nombre, n):
        super().__init__(nombre)
        self.n = n

    def leer(self):
        return list(range(self.n))

In [ ]:
# Ambas se usan igual: .leer()
fuentes = [
    FuenteTexto("csv", "nombre,edad\nAna,25\nLuis,30"),
    FuenteNumerica("secuencia", 5)
]

for f in fuentes:
    print(f"{f} → {f.leer()}")

# El padre define el contrato (.leer()), los hijos lo implementan.
# Código que trabaja con FuenteDatos funciona con CUALQUIER hijo.

In [ ]:
# ¿Qué pasa si llamas .leer() en el padre directamente?
try:
    base = FuenteDatos("abstracta")
    base.leer()
except NotImplementedError as e:
    print(f"Error esperado: {e}")
    print("→ El padre obliga a los hijos a implementar .leer()")

### ¿Herencia o composición?

- **Herencia** = "X **es un tipo de** Y" → `FuenteCSV` **es** una `FuenteDatos`
- **Composición** = "X **tiene** un Y" → un `Carro` **tiene** un `Motor`

Si la relación no es "es un tipo de", probablemente no necesitas herencia.

---
## 3. Documentación

Los docstrings son documentación que **vive dentro del código** y se accede con `help()`.

In [ ]:
def buscar(texto, patron, ignorar_caso=False):
    """Busca un patrón en el texto y retorna las coincidencias.

    Args:
        texto: El string donde buscar.
        patron: El substring a buscar.
        ignorar_caso: Si True, no distingue mayúsculas.

    Returns:
        list[str]: Líneas que contienen el patrón.
    """
    lineas = texto.split("\n")
    if ignorar_caso:
        return [l for l in lineas if patron.lower() in l.lower()]
    return [l for l in lineas if patron in l]

# help() muestra el docstring formateado
help(buscar)

In [ ]:
# Probémosla
texto = """Python es genial
python es flexible
Java es verboso
PYTHON ES RÁPIDO"""

print("Con case: ", buscar(texto, "Python"))
print("Sin case:", buscar(texto, "Python", ignorar_caso=True))

In [ ]:
# dir() lista todo lo que un objeto tiene
print("Métodos públicos de Contador:")
print([m for m in dir(Contador) if not m.startswith("_")])

print("\nTodo (incluyendo internos):")
print([m for m in dir(Contador) if m.startswith("_") and not m.startswith("__")])

**Regla**: Si escribes buenos docstrings en Google style (`Args`, `Returns`, `Raises`), `help()` se convierte en la documentación de tu librería gratis.

---
## 4. Testing

### doctest: documentación que se verifica

In [ ]:
def factorial(n):
    """Calcula el factorial de n.

    >>> factorial(0)
    1
    >>> factorial(5)
    120
    >>> factorial(1)
    1
    """
    if n <= 1:
        return 1
    return n * factorial(n - 1)

# Ejecutar los doctests
import doctest
resultados = doctest.testmod(verbose=True)
print(f"\n{resultados.attempted} tests, {resultados.failed} fallos")

Los `>>>` dentro del docstring son ejemplos que Python puede **ejecutar y verificar**. Si el resultado no coincide, el test falla.

### pytest: tests organizados

En un notebook no podemos ejecutar `pytest` directamente, pero podemos ver la estructura y simular la lógica.

In [ ]:
# Así se ve un archivo de tests para pytest:
#
# tests/test_operaciones.py
#
# from mi_paquete import sumar, restar
#
# def test_sumar_positivos():
#     assert sumar(2, 3) == 5
#
# def test_sumar_negativos():
#     assert sumar(-1, -1) == -2
#
# def test_restar():
#     assert restar(10, 3) == 7

# Podemos simular lo mismo aquí:
from mi_paquete import sumar, restar

def test_sumar_positivos():
    assert sumar(2, 3) == 5

def test_sumar_negativos():
    assert sumar(-1, -1) == -2

def test_restar():
    assert restar(10, 3) == 7

# Ejecutar
for test_fn in [test_sumar_positivos, test_sumar_negativos, test_restar]:
    try:
        test_fn()
        print(f"  ✓ {test_fn.__name__}")
    except AssertionError as e:
        print(f"  ✗ {test_fn.__name__}: {e}")

### Convenciones de pytest

| Regla | Ejemplo |
|-------|---------|
| Archivos empiezan con `test_` | `test_operaciones.py` |
| Funciones empiezan con `test_` | `def test_sumar_positivos():` |
| Usa `assert` para verificar | `assert resultado == esperado` |
| Un test = un comportamiento | No metas 5 asserts en un test |
| El nombre dice qué verifica | `test_sumar_negativos`, no `test_1` |

In [ ]:
# Para ejecutar pytest en la terminal:
# pytest tests/                      ← ejecuta todos los tests
# pytest tests/test_operaciones.py   ← ejecuta un archivo
# pytest -v                          ← modo verbose (más detalle)

# Vamos a crear un archivo de test real para verificar
os.makedirs("tests", exist_ok=True)
with open("tests/test_operaciones.py", "w") as f:
    f.write('''
from mi_paquete import sumar, restar

def test_sumar_positivos():
    assert sumar(2, 3) == 5

def test_sumar_negativos():
    assert sumar(-1, -1) == -2

def test_restar():
    assert restar(10, 3) == 7

def test_sumar_flotantes():
    assert abs(sumar(0.1, 0.2) - 0.3) < 1e-9
''')

print("Archivo de test creado: tests/test_operaciones.py")
print("Para ejecutar: pytest tests/ -v")

---
## Resumen

| Concepto | Lo esencial |
|----------|------------|
| **Paquete** | Directorio + `__init__.py`. Exporta lo que el usuario necesita. |
| **Clase** | Agrupa datos + comportamiento. Usa cuando hay estado compartido. |
| **Herencia** | "X es un tipo de Y". El padre define el contrato, los hijos implementan. |
| **Docstrings** | Google style: `Args`, `Returns`, `Raises`. `help()` los muestra gratis. |
| **pytest** | `test_` en nombre de archivo y función. Un test = un comportamiento. |
| **doctest** | Ejemplos `>>>` en el docstring que se verifican automáticamente. |

Si algo de esto no te queda claro, revisa el [material de texto](../01_repaso_ingenieria.md) o vuelve a la sección correspondiente del curso de DataCamp.

---
## Ejercicio: mini-paquete

Crea un paquete `calculadora` con:
- `basica.py`: `sumar`, `restar`, `multiplicar`, `dividir`
- `estadistica.py`: `promedio`, `mediana`
- `__init__.py`: exporta todo
- Docstrings en cada función
- Al menos un test por función

In [ ]:
# Tu código aquí

